In [20]:
import os
import gzip
import numpy as np
import json
import networkx as nx

from utilities import qubo_to_ising, matrix_to_couplings

In [22]:
Ntot = 100

# Generate random 3Reg graph
G = nx.random_regular_graph(3, Ntot)
edges = list(G.edges)

# assign unity edge weights
weights = {edge: 1 for edge in edges}
nx.set_edge_attributes(G, weights, "weight")

# Create MaxCut QUBO
Q = np.zeros((Ntot, Ntot))
for (i, j) in edges:
    Q[i, j] = weights[(i, j)]
    Q[i, i] -= weights[(i, j)]
    Q[j, j] -= weights[(i, j)]

J, h = matrix_to_couplings(Q)

# Convert to Ising
J_ising, h_ising, offset = qubo_to_ising(Q)

# Save the graph as JSON
# Save graph
with open(f"../data/MaxCut/instances/graph0.json", "w") as f:
    # edge and weight
    json.dump({"N": Ntot, "edges": {f"{k[0]},{k[1]}": v for k, v in weights.items()}}, f, indent=4)

# Save QUBO
with open(f"../data/MaxCut/QUBO/qubo_graph0.json", "w") as f:
    json.dump({"J": {f"{k[0]},{k[1]}": v for k, v in J.items()}, "h": {str(k): v for k, v in h.items()}, "c": offset}, f, indent=4)

# Save Ising
with open(f"../data/MaxCut/Ising/ising_graph0.json", "w") as f:
    json.dump({"J": {f"{k[0]},{k[1]}": v for k, v in J_ising.items()}, "h": {str(k): v for k, v in h_ising.items()}}, f, indent=4)   